# Gaplock Plot Explorer
This notebook loads the CSV/JSON artifacts we dump under `plots/<scenario>/<run>/` and renders attacker/defender paths or heat maps.
1. Run training to produce `paths_<run>.csv` + `run_config_<run>.json`.
2. Point `run_dir` below to the folder that contains those files.
3. Execute the plotting cells to visualize trajectories.

In [1]:
from pathlib import Path
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Set this to the directory containing paths_*.csv and run_config_*.json (e.g., plots/<scenario>/<run>/<timestamp>)
run_dir = Path('plots') / 'gaplock_td3'
if not run_dir.exists():
    raise FileNotFoundError(f'{run_dir} does not exist')
csv_candidates = sorted(run_dir.glob('paths_*.csv'))
json_candidates = sorted(run_dir.glob('run_config_*.json'))
if not csv_candidates or not json_candidates:
    raise FileNotFoundError('No paths/run_config files found under ' + str(run_dir))
paths_csv = csv_candidates[-1]
config_json = json_candidates[-1]
print('Using', paths_csv)
print('Config', config_json)
df = pd.read_csv(paths_csv)
with open(config_json) as fh:
    run_cfg = json.load(fh)
run_cfg


NameError: name 'csv_candidates' is not defined

In [ ]:
episode = int(df['episode'].min())
agent = 'car_0'
sel = df[(df['episode'] == episode) & (df['agent_id'] == agent)]
plt.figure(figsize=(6,6))
plt.plot(sel['x'], sel['y'], label=f'{agent} ep {episode}')
plt.axis('equal')
plt.legend()
plt.title('Trajectory')
plt.show()

In [ ]:
agent = 'car_0'
agent_df = df[df['agent_id'] == agent]
plt.figure(figsize=(6,6))
sns.kdeplot(data=agent_df, x='x', y='y', fill=True, cmap='magma', thresh=0.05)
plt.axis('equal')
plt.title(f'Heat map for {agent}')
plt.show()

In [ ]:
def plot_stacked_paths(dataframe, agents=None, max_episodes=None, stride=1):
    plt.figure(figsize=(8,8))
    agents = agents or dataframe['agent_id'].unique()
    for agent in agents:
        subset = dataframe[dataframe['agent_id'] == agent]
        episodes = sorted(subset['episode'].unique())[::stride]
        if max_episodes:
            episodes = episodes[:max_episodes]
        for ep in episodes:
            ep_df = subset[subset['episode'] == ep]
            plt.plot(ep_df['x'], ep_df['y'], alpha=0.1, label=f'{agent}' if ep == episodes[0] else '')
    plt.axis('equal')
    plt.legend()
    plt.title('Stacked Trajectories')
    plt.show()
plot_stacked_paths(df, agents=['car_0','car_1'])


In [ ]:
def plot_cause_histogram(dataframe):
    plt.figure(figsize=(6,4))
    dataframe['cause_code'].value_counts().sort_index().plot(kind='bar')
    plt.xlabel('Cause code (0=success,1=attacker crash,2=both crash,3=truncation,4=other)')
    plt.ylabel('Episodes')
    plt.title('Episode termination causes')
    plt.show()
plot_cause_histogram(df)


In [ ]:
import yaml
def load_map_geometry(run_config):
    try:
        map_name = run_config['scenario']['env']['map']
    except Exception:
        return None
    map_path = Path('maps') / map_name / 'map.yaml'
    if not map_path.exists():
        return None
    with map_path.open() as fh:
        data = yaml.safe_load(fh)
    return data
map_bundle = load_map_geometry(run_cfg)
map_bundle


In [ ]:
def plot_paths_with_map(dataframe, map_data, agents=('car_0','car_1'), episode=None):
    plt.figure(figsize=(8,8))
    if map_data and 'walls' in map_data:
        for wall in map_data['walls']:
            xs = [pt[0] for pt in wall]
            ys = [pt[1] for pt in wall]
            plt.plot(xs, ys, color='gray', linewidth=1)
    for agent in agents:
        subset = dataframe[dataframe['agent_id'] == agent]
        if episode is not None:
            subset = subset[subset['episode'] == episode]
        plt.plot(subset['x'], subset['y'], label=agent)
    plt.axis('equal')
    plt.legend()
    plt.title('Paths over map')
    plt.show()
plot_paths_with_map(df, map_bundle)
